##0: General Concepts

Artifical Intelligence is defined as a computer system able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between langauges. These tasks can be achieved in multiple ways however a popular one is the machine learning sub-area of AI. This is because the programs in this sub-area are capable of self-adjusting and getting better with minimal human interaction. 
With Machine Learning you can look at three different types of learning which are supervised, unsupervised, and reinforcement. Supervised learning is when the system learns based off of pre-categorized data. This allows the system to look back and adjust its parameters to increase its accuracy. This is a great method when humans are able to easily make large datasets to train with but need to automate the process with a high accuracy. In Unsupervised Learning the data is not labeled so the system has the job of finding relevant data to categorize by. This is a useful technique when humans are not sure about the specific similarities between a dataset and want to determine new ways to categorize a set of data. Then there is reinforcement learning which attempts to find a model which given any set of data can force it into the desired outcome. This type of learning is more likely to be used in other systems such as video games in order to determine the best strategies to succeed.

## 1: Building a model

To address how building a model works we will look at the pieces of a convolutional neural network. This type of network is considered to be under the Deep Learning subsection of Machine Learning. It is used on Images mainly which are two dimensional and have three channels of color for each pixel. This data is passed through the model to extract features and determine whcih features are the most important.
The first piece of the CNN we will look at is the convolution filter. This filter is generally a 3x3 or 5x5 matrix which slides over the image and using the data and matrix to output a lower resolution feature map. Below we can see what adding a layer of convolution looks like.


In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))

This is a 3x3 convolution layer which will extract features from the input image of dimensions 150x150x3. The output feature map will be of smaller dimensions due to its reducing nature. Adding this layer to the model gives it the ability to begin finding important features which will later help in classifying the data.
The next piece of the model we wil look at is the Max Pooling Layers. The goal behind these layers is that they will pull the most important information from the conolutions output feature map and produce a smaller feature map. The goal of this is to save time in later steps by elimainating the less important information that the convoilutional layer may have produced. Adding a Pooling layer to the current model would look like the following.

In [0]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))

This max pooling layer will take the feature map produced by the convolution above it and make sure that any following layers of convolution are not spending excess time doing computations of not so important data.
Once these two pieces have been put together in some manner of mixing and matching the following layers will be a set of fully connected layers. This means that every node within the layer is connected with everynode of the following layer. These nodes will take in the feature data gathered during the convolution and pooling stages and begin to use them as a way to classify the data. These layers will be added to the model as seen in the following snipet of code.

In [0]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

On lines 5 and 6 of the code above we can see the fully connected layers. These layers will manipulate the feature data to attempt to determine what the data represents. The final layer represents the output layer where in this case the data is classifed in a binary fashion.
Overall this is a very simple model however due to the effectiveness of CNN's it would not be completely ineffective however it is not nearly as powerful as other networks that have been built. The complexity of a model can vary by large amounts however the internal pieces of a CNN always has these key pieces.

##2: Compiling a model

After a model is built with all of its layers the next step is compiling it. While similar to compiling any other piece of software to make it capable of working in these cases the most important part of this step is setting up certain hyperparameters such as optimizers and learning rates. For this section we will be analyzing hte following code.

In [0]:
model.compile(
    loss='binary_crossentropy', 
    optimizer=optimizers.RMSprop(lr=2e-5), 
    metrics=['acc'])

The aspects we will focus on is all on line number 3. It contains the optimizer and the initial learning rate. First we will look at the learning rate which is exactly what it sounds like it is the variable which detemrines how drastic each iteration of the model changes. In the case of this model it is being set to a learning rate of 2e-5. This is a fairly standard number to begin with. The goal of the learning rate is to make sure that the model does not skip over the optimal model with maximum accuracy and minimal loss. 
Now for the more complicated part is the optimizer. In this code the optimized that is being used is called RMSprop. The main property of this optiumized is that it changes the learning rate of the model depening on the current progress being made by the model. Other optimizers such as Stochastic gradient descent take differnet approaches to optimizing the model to get a better accuracy and less loss. This part of producing a model is not very code heavy but can have a significant effect on how well the model does after trainign has begun.

## 3. Training a model

Training the model built is the process of running through data and letting the model adjust its feature weights in order to create the best model possible. In the cas eof a CNN the training of the model takes the form of the following code.

In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

In the case above train_generator and validation_generator are used to feed the model data. The steps_per_epoch is the numbe of data points to go through before adjusting the model. Epochs is the number of times the model will be trained and asjusted before its final iteration. Finally validation_steps is the number of data points to test the model on between epochs.
These values are the parameters that determine how long the model has to imporve. How the data is given to the model and how many iterations and time between iterations there is determines how well the model can classify data. One of the most important parts of training is to make sure that the model does not over or underfit the datasets. If it is overfitting it is doing significantly worse at predicting the validatiuon data than the test data and if underfitting the model is not able to classify either dataset well. The goal of trianing the model is to achieve a model which has nearly identifcal accuracy and loss statistics to each other. When a low steps_per_epochs and epochs value is set it can cause significant underfitting as the model is unable to look at enough data to begin to understand its tendencies causing it to underfit. On the other hand if the data passed to the model is very similar and not random the model could begin to overfit due to the model becoming to well adjusted to the training dataset. Methods of fixing this are increasing unique data exposure by augmenting the data for overfitting and making sure that the model has enough data to learn in underfitting.

##4: Finetuning a Pretrained model

Once a model has been trained and has achieved a reasonable accuracy and loss it can then move onto 
the stage of finetuning. The idea behind fine tunign is to give each layer in the network time to find its best weighting for each of the features it is evaluating by not allowing layers before it to adjust. To do this we use code like the following.

In [0]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In the code above we are only setting the trainable layers to be the one named block5_conv1 and any following layers. The idea behind this is that if the weighting preceding layer block5_conv1 is frozen then block5_conv1 is going to have the most significant impact with its changes due to it being the first layer which the data is exposed to new weights. While it is not neccessarily true that the first layer to manipulate the data has the largest effect on the outcome of the model it is generally true. So by changing which layers are able to adjust their weighting they become more senstive to more features and begin to better identify what each piece means. 
The goal of finetuning is to allow the model to take a more precise approach to adusting itself by not allowing it to make changes to every node at once forcing it to make sure that every node has a weighting that more correctly estimates the final outcome rather than changing all nodes without allowing the later layers to become more sensitive to each feature.